In [111]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.utils import to_categorical 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('2016-forecast-cnn.csv')
df = df.drop(['Unnamed: 0','No','NIM'],axis=1)
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,3.5,3.5,4.0,3.5,3.5,4.0,3.5,3.5,3.5,1
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,3.5,4.0,3.5,3.5,3.0,3.5,4.0,3.5,3.5,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,3.0,3.5,3.5,4.0,3.5,3.5,3.5,2.5,3.5,1
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,2.5,2.5,3.5,3.5,2.5,2.0,1.0,4.0,4.0,1


In [112]:
df2 = pd.read_csv('2016 data v5.csv')
df2.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,KSI,Pilihan 1,Pilihan 2,Peminatan
0,1,1202160002,4.0,3.5,0.0,4.0,3.5,4.0,3.5,3.5,...,4.0,3.5,3.5,4.0,3.5,3.5,0.0,ERP,EDM,ERP
1,2,1202160009,4.0,3.5,0.0,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.0,3.5,4.0,3.5,0.0,ERP,EDM,ERP
2,3,1202160010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0,NaN
3,4,1202160011,4.0,4.0,0.0,3.5,3.5,4.0,3.0,4.0,...,3.5,4.0,3.5,3.5,3.5,2.5,0.0,ERP,EDM,ERP
4,5,1202160013,3.5,3.0,0.0,3.5,2.0,2.0,2.5,4.0,...,3.5,3.5,2.5,2.0,1.0,4.0,0.0,EA,EDM,EDM


In [113]:
df['Pilihan 1'] = df2['Pilihan 1']
df['Pilihan 2'] = df2['Pilihan 2']
df['Peminatan'] = df2['Peminatan']
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,ERP,EDM,ERP
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,ERP,EDM,ERP
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,NaN,0,NaN
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,ERP,EDM,ERP
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,EA,EDM,EDM


In [114]:
new_class = [0] * len(df['Peminatan'])
for index, row in df.iterrows() :
    if row["Peminatan"] == "TECHNO" :
        new_class[index] = 1
    elif row['Peminatan'] == "EAD" :
        new_class[index] = 2
    elif row['Peminatan'] == "EDM" :
        new_class[index] = 3
    elif row['Peminatan'] == "ERP" :
        new_class[index] = 4
    elif row['Peminatan'] == "EA" :
        new_class[index] = 5
    elif row['Peminatan'] == "EIM" :
        new_class[index] = 6
    elif row['Peminatan'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Peminatan"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,ERP,EDM,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,ERP,EDM,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,NaN,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,ERP,EDM,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,EA,EDM,3


In [115]:
new_class = [0] * len(df['Pilihan 1'])
for index, row in df.iterrows() :
    if row["Pilihan 1"] == "TECHNO" :
        new_class[index] = 1
    elif row['Pilihan 1'] == "EAD" :
        new_class[index] = 2
    elif row['Pilihan 1'] == "EDM" :
        new_class[index] = 3
    elif row['Pilihan 1'] == "ERP" :
        new_class[index] = 4
    elif row['Pilihan 1'] == "EA" :
        new_class[index] = 5
    elif row['Pilihan 1'] == "EIM" :
        new_class[index] = 6
    elif row['Pilihan 1'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Pilihan 1"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,4,EDM,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,4,EDM,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,0,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,4,EDM,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,5,EDM,3


In [116]:
new_class = [0] * len(df['Pilihan 2'])
for index, row in df.iterrows() :
    if row["Pilihan 2"] == "TECHNO" :
        new_class[index] = 1
    elif row['Pilihan 2'] == "EAD" :
        new_class[index] = 2
    elif row['Pilihan 2'] == "EDM" :
        new_class[index] = 3
    elif row['Pilihan 2'] == "ERP" :
        new_class[index] = 4
    elif row['Pilihan 2'] == "EA" :
        new_class[index] = 5
    elif row['Pilihan 2'] == "EIM" :
        new_class[index] = 6
    elif row['Pilihan 2'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Pilihan 2"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,4,3,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,4,3,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,0,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,4,3,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,5,3,3


In [117]:
mainData = df[df.Peminatan != 0]
mainData = mainData.drop(['Lulus Tepat Waktu'],axis=1)
target = ["Peminatan"]
predictors = list(set(list(mainData.columns))-set(target))
mainData[predictors] = mainData[predictors]/mainData[predictors].max()
mainData[predictors].head()

,SCM,Pilihan 1,Desjar,Alpro,EA,OOP,Matdis,WEB,Manprosi,Basdat,...,PO,PI,Sisop,Statistik,Manjarkom,APSI,Pilihan 2,MSDM,RPB,Strukdat
0,0.875,0.571429,0.875,0.875,1.000,1.000,0.875,1.000,0.875,0.875,...,0.875,1.000,1.000,0.875,0.875,0.875,0.428571,0.875,0.875,0.875
1,1.000,0.571429,0.875,1.000,0.875,1.000,0.875,0.875,0.875,0.875,...,0.875,0.875,0.875,0.875,0.750,0.875,0.428571,1.000,0.875,0.875
3,0.875,0.571429,0.625,0.875,0.875,1.000,1.000,1.000,0.875,1.000,...,0.750,0.875,0.875,0.750,0.875,0.875,0.428571,0.875,1.000,0.750
4,0.625,0.714286,1.000,0.875,0.875,0.875,1.000,0.500,1.000,0.875,...,0.625,0.875,0.500,0.625,0.625,0.500,0.428571,0.250,0.750,0.500
5,0.875,1.000000,0.625,0.875,0.875,0.250,1.000,0.750,0.625,0.625,...,0.625,0.875,0.500,0.500,0.250,0.875,0.428571,0.875,0.250,0.750


In [118]:
X = mainData[target]
y = mainData[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=40)
print(X_train.shape)
print(y_train.shape)
print(X_test.head())
print(y_test.shape)

(184, 1)
(184, 21)
     Peminatan
304          7
384          3
319          7
289          3
245          3
(123, 21)


In [119]:
X_trains = to_categorical(X_train)
X_tests = to_categorical(X_test)

print(X_trains.shape)

(184, 8)


In [120]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=21))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [121]:
model.fit(y_train, X_trains, epochs=200)

Epoch 1/200
184/184 [==============================] - 0s 444us/step - loss: 2.1138 - accuracy: 0.1196
Epoch 2/200
184/184 [==============================] - 0s 54us/step - loss: 2.0375 - accuracy: 0.2065
Epoch 3/200
184/184 [==============================] - 0s 76us/step - loss: 2.0015 - accuracy: 0.3098
Epoch 4/200
184/184 [==============================] - 0s 60us/step - loss: 1.9628 - accuracy: 0.2989
Epoch 5/200
184/184 [==============================] - 0s 65us/step - loss: 1.9224 - accuracy: 0.3152
Epoch 6/200
184/184 [==============================] - 0s 65us/step - loss: 1.8767 - accuracy: 0.4022
Epoch 7/200
184/184 [==============================] - 0s 70us/step - loss: 1.8296 - accuracy: 0.3859
Epoch 8/200
184/184 [==============================] - 0s 65us/step - loss: 1.7757 - accuracy: 0.4022
Epoch 9/200
184/184 [==============================] - 0s 54us/step - loss: 1.7199 - accuracy: 0.4402
Epoch 10/200
184/184 [==============================] - 0s 54us/step - loss: 1.65

In [122]:
pred_train= model.predict(y_train)
scores = model.evaluate(y_train, X_trains, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(y_test)
scores2 = model.evaluate(y_test, X_tests, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on training data: 1.0% 
 Error on training data: 0.0
Accuracy on test data: 0.869918704032898% 
 Error on test data: 0.13008129596710205


In [123]:
dataTesting = pd.read_csv('data_nilai_mahasiswa.csv')
dataTesting.head()

,id,nim,nama,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,...,skor_peminatan_1,id_peminatan_2,skor_peminatan_2,peminatan,status,prioritas,skor_buang,status_lulus,created_at.1,updated_at.1
0,456,1202170258,PUTU DELA APSARI MERTADEWI,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,19.0,3,16.0,4,10,0,0.0,NaN,2020-01-01 18:58:20,2020-03-13 14:16:04
1,603,1202174065,MUHAMMAD FIKRI AZHAR,3.5,3.0,3.5,3.5,3.5,2.0,3.0,...,13.0,6,12.5,2,10,0,0.0,NaN,2020-01-01 19:00:11,2020-03-13 14:16:04
2,677,1202174208,YUNI ARIYANI ARIFIN,3.5,3.0,3.5,4.0,3.5,3.0,2.5,...,13.0,3,14.0,3,9,0,0.0,NaN,2020-01-01 19:05:00,2020-03-13 14:16:04
3,741,1202174372,IDELLIA MUTHIA NURBAISA,4.0,3.5,4.0,4.0,4.0,4.0,3.5,...,16.5,6,14.5,2,10,0,0.0,NaN,2020-01-01 19:11:34,2020-03-13 14:16:04
4,430,1202170180,IBRAM MOSZARDO,2.5,3.5,3.0,2.0,3.5,3.0,2.0,...,12.5,7,12.0,3,10,0,0.0,NaN,2020-01-01 19:11:53,2020-03-13 14:16:04


In [124]:
dataTesting.rename(columns={'id_peminatan_1':'Pilihan 1','id_peminatan_2':'Pilihan 2'}, inplace=True)
newDataTesting = dataTesting.drop(columns={
    "id",
    "skor_peminatan_1",
    "skor_peminatan_2",
    "status","prioritas",
    "skor_buang",
    'created_at.1',
    'updated_at.1',
    'created_at',
    'updated_at',
    'id.1',
    'id_users',
    'status_lulus',
    'peminatan'
},axis=1)
newDataTesting.head()

,nim,nama,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,...,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Pilihan 1,Pilihan 2
0,1202170258,PUTU DELA APSARI MERTADEWI,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,3.5,4.0,3.0,3.5,4,3
1,1202174065,MUHAMMAD FIKRI AZHAR,3.5,3.0,3.5,3.5,3.5,2.0,3.0,3.5,...,3.5,3.5,3.0,2.0,4.0,3.0,2.0,3.5,2,6
2,1202174208,YUNI ARIYANI ARIFIN,3.5,3.0,3.5,4.0,3.5,3.0,2.5,4.0,...,0.0,3.5,3.5,2.0,3.0,3.5,3.0,3.5,6,3
3,1202174372,IDELLIA MUTHIA NURBAISA,4.0,3.5,4.0,4.0,4.0,4.0,3.5,4.0,...,4.0,3.5,3.5,3.0,4.0,3.5,3.0,4.0,2,6
4,1202170180,IBRAM MOSZARDO,2.5,3.5,3.0,2.0,3.5,3.0,2.0,3.5,...,2.0,3.5,3.5,3.0,3.5,4.0,3.0,2.5,3,7


In [125]:
dataTestingPredictor = newDataTesting[predictors]
dataTestingHasil = model.predict_classes(dataTestingPredictor)

newDataTesting["Peminatan"] = dataTestingHasil
newDataTesting.rename(columns={'nim':'NIM'}, inplace=True)
newDataTesting.drop(columns="nama",axis=1).head()

,NIM,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,...,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Pilihan 1,Pilihan 2,Peminatan
0,1202170258,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,3.5,4.0,3.0,3.5,4,3,7
1,1202174065,3.5,3.0,3.5,3.5,3.5,2.0,3.0,3.5,3.5,...,3.5,3.0,2.0,4.0,3.0,2.0,3.5,2,6,3
2,1202174208,3.5,3.0,3.5,4.0,3.5,3.0,2.5,4.0,3.5,...,3.5,3.5,2.0,3.0,3.5,3.0,3.5,6,3,7
3,1202174372,4.0,3.5,4.0,4.0,4.0,4.0,3.5,4.0,4.0,...,3.5,3.5,3.0,4.0,3.5,3.0,4.0,2,6,3
4,1202170180,2.5,3.5,3.0,2.0,3.5,3.0,2.0,3.5,3.5,...,3.5,3.5,3.0,3.5,4.0,3.0,2.5,3,7,7


In [ ]:
newDataTesting.to_csv('201-forecast-cnn.csv')